In [21]:
import os
from docx import Document

# %%
def contains_table(docx_path):
    """Kiểm tra file DOCX có chứa bảng không"""
    try:
        doc = Document(docx_path)
        return len(doc.tables) > 0
    except Exception as e:
        print(f"Lỗi khi đọc file {docx_path}: {e}")
        return False

# %%
def find_docx_with_tables(root_folder):
    """Tìm và in các file DOCX có chứa bảng"""
    results = []
    
    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower().endswith('.docx'):
                full_path = os.path.join(foldername, filename)
                
                if contains_table(full_path):
                    results.append(full_path)
                    print(f"Phát hiện bảng: {full_path}")
    
    return results

root_directory = "./data"  # THAY ĐỔI ĐƯỜNG DẪN NÀY
found_files = find_docx_with_tables(root_directory)
if found_files:
    for i, path in enumerate(found_files, 1):
        print(f"{i}. {path}")
else:
    print("Không tìm thấy file DOCX nào chứa bảng")


Không tìm thấy file DOCX nào chứa bảng


In [ ]:
import os
import re
import time
from pathlib import Path
from docx import Document
from together import Together
from dotenv import load_dotenv


# Nạp biến môi trường từ file .env
load_dotenv()
api_key = os.getenv("TOGETHER_API_KEY")

client = Together(api_key=api_key)


# Prompt để chuyển đổi sang Markdown
CONVERSION_PROMPT = """
## Mục tiêu
Chuyển đổi tài liệu Word (.docx) sang định dạng Markdown theo template chuẩn tối ưu cho hệ thống RAG (Retrieval-Augmented Generation).

## Yêu cầu định dạng đầu ra

### Template chuẩn:
```markdown
# [TIÊU ĐỀ CHÍNH]

## [Mục lớn 1]
### [Tiểu mục 1.1]
- Nội dung chính...
- **Lưu ý**: Chi tiết quan trọng

### [Tiểu mục 1.2]
```code
Mã lệnh/command
```

## [Mục lớn 2]
| Cột 1 | Cột 2 | Mô tả |
|-------|-------|--------|
| A     | 100   | Chi tiết A |
| B     | 200   | Chi tiết B |
```

## Quy tắc chuyển đổi

### 1. Cấu trúc heading
- Tiêu đề chính: H1 (`#`)
- Heading cấp 1 trong Word → H2 (`##`)
- Heading cấp 2 trong Word → H3 (`###`)
- Heading cấp 3 trong Word → H4 (`####`)

### 2. Xử lý văn bản
- Giữ nguyên bold cho **thuật ngữ quan trọng**
- Chuyển italic thành _emphasis_
- Gạch dưới → **bold** nếu quan trọng, ngược lại bỏ
- Chữ màu/kích thước → bỏ định dạng

### 3. Danh sách
- List có số → đánh số thứ tự
- List không số → dấu gạch đầu dòng (`-`)
- List đa cấp: thụt lề 2 spaces cho mỗi cấp

### 4. Bảng biểu
```markdown
| Header 1 | Header 2 |
|----------|----------|
| Cell 1   | Cell 2   |
```
- Giữ nguyên số cột/hàng
- Căn trái mặc định

### 5. Hình ảnh
- Chuyển thành: `![Mô tả](path/to/image.jpg)`
- Lấy alt text từ Word, nếu không có → ghi "Hình minh họa"

### 6. Mã lệnh/cấu hình
Đặt trong code block với ngôn ngữ phù hợp:
```python
print("Hello World")
```

### 7. Liên kết
- Chuyển thành `[văn bản hiển thị](URL)`
- Nếu là email: `[tên](mailto:email@domain.com)`

## Xử lý đặc biệt
- Section ngăn cách: `---` (3 dấu gạch ngang)
- Ghi chú: `> Ghi chú quan trọng`
- Thuật ngữ: `**Thuật ngữ**: Định nghĩa`

## Yêu cầu chất lượng
- Không làm mất thông tin gốc
- Giữ nguyên cấu trúc phân cấp
- Đảm bảo cấu trúc phân cấp rõ ràng
- Giữ nguyên ngữ nghĩa nội dung, không thay đổi ý nghĩa

"""

def validate_markdown_structure(content):
    """Kiểm tra cấu trúc markdown"""
    errors = []
    
    # Kiểm tra tiêu đề chính
    if not re.search(r'^#\s+.+', content, re.MULTILINE):
        errors.append("Thiếu tiêu đề chính (H1)")
    
    # Kiểm tra tỷ lệ heading hợp lý
    h2_count = len(re.findall(r'^##\s+', content, re.MULTILINE))
    h3_count = len(re.findall(r'^###\s+', content, re.MULTILINE))
    
    if h3_count > 0 and h2_count == 0:
        errors.append("Có H3 nhưng thiếu H2")
    
    # Kiểm tra code block đóng mở đúng
    code_blocks = re.findall(r'```', content)
    if len(code_blocks) % 2 != 0:
        errors.append("Code block chưa đóng")
    
    return errors if errors else "Cấu trúc hợp lệ"

def extract_text_from_docx(docx_path):
    """Trích xuất text từ file DOCX"""
    try:
        doc = Document(docx_path)
        full_text = []
        
        for para in doc.paragraphs:
            text = para.text.strip()
            if text:
                full_text.append(text)
        
        content = "\n".join(full_text)
        return content
    except Exception as e:
        print(f"❌ Lỗi khi đọc file {docx_path}: {e}")
        return None

def convert_to_markdown_with_together(content, max_retries=3):
    """Chuyển đổi nội dung sang Markdown sử dụng Together AI"""
    
    for attempt in range(max_retries):
        try:
            print(f"🔄 Đang gọi Together AI (lần thử {attempt + 1}/{max_retries})...")
            
            response = client.chat.completions.create(
                model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
                messages=[
                    {"role": "system", "content": CONVERSION_PROMPT},
                    {"role": "user", "content": content}
                ],
                temperature=0
            )
            
            markdown_content = response.choices[0].message.content
            
            # Kiểm tra chất lượng markdown
            validation_errors = validate_markdown_structure(markdown_content)
            
            if isinstance(validation_errors, list) and len(validation_errors) > 0:
                print(f"⚠️  Lần thử {attempt + 1}: Markdown chưa hợp lệ - {validation_errors}")
                if attempt < max_retries - 1:
                    time.sleep(2)  # Đợi 2 giây trước khi thử lại
                    continue
                else:
                    print("⚠️  Đã thử tối đa, sử dụng kết quả cuối cùng")
            else:
                print(f"✅ Lần thử {attempt + 1}: Markdown hợp lệ!")
                return markdown_content
            
        except Exception as e:
            print(f"❌ Lỗi khi gọi Together AI (lần thử {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            else:
                print("❌ Đã thử tối đa, không thành công")
                return None
    
    return markdown_content if 'markdown_content' in locals() else None

def find_all_docx_files(folder_path):
    """Tìm tất cả file DOCX trong folder và subfolder"""
    docx_files = []
    folder_path = Path(folder_path)
    
    print(f"🔍 Đang tìm file DOCX trong: {folder_path}")
    
    for file_path in folder_path.rglob("*.docx"):
        # Bỏ qua file tạm của Office (bắt đầu bằng ~$)
        if not file_path.name.startswith("~$"):
            docx_files.append(file_path)
            print(f"📄 Tìm thấy: {file_path}")
    
    print(f"📊 Tổng cộng tìm thấy {len(docx_files)} file DOCX")
    return docx_files

def generate_output_path(docx_path, input_folder, output_folder):
    """Tạo đường dẫn output tương ứng với cấu trúc folder và lưu tên file dạng 'đường_dẫn_tên_file.md'"""
    docx_path = Path(docx_path)
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    
    # Tính toán đường dẫn tương đối
    relative_path = docx_path.relative_to(input_folder)
    
    # Chuyển đường dẫn tương đối thành đường dẫn chuỗi
    relative_path_str = str(relative_path).replace(os.sep, '_')
    
    # Tạo đường dẫn output với tên file dạng đường_dẫn_tên_file.md
    output_path = output_folder / f"{relative_path_str}.md"
    
    return output_path


def process_docx_files(input_folder, output_folder):
    """Xử lý tất cả file DOCX trong folder"""
    
    # Tạo thư mục output nếu chưa tồn tại
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    
    # Tìm tất cả file DOCX
    docx_files = find_all_docx_files(input_folder)
    
    if not docx_files:
        print("❌ Không tìm thấy file DOCX nào!")
        return
    
    successful_conversions = 0
    failed_conversions = 0
    
    for i, docx_file in enumerate(docx_files, 1):
        print(f"\n{'='*60}")
        print(f"🔄 Đang xử lý file {i}/{len(docx_files)}: {docx_file.name}")
        print(f"{'='*60}")
        
        # Tạo đường dẫn output
        output_path = generate_output_path(docx_file, input_folder, output_folder)
        
        # Tạo thư mục con nếu cần
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        print(f"📂 Đường dẫn đầu vào: {docx_file}")
        print(f"📁 Đường dẫn đầu ra: {output_path}")
        
        # Trích xuất nội dung từ DOCX
        print("📖 Đang đọc nội dung DOCX...")
        content = extract_text_from_docx(docx_file)
        
        if content is None:
            print("❌ Không thể đọc file DOCX")
            failed_conversions += 1
            continue
        
        print(f"📝 Đã đọc {len(content)} ký tự")
        
        # Chuyển đổi sang Markdown
        print("🤖 Đang chuyển đổi sang Markdown...")
        markdown_content = convert_to_markdown_with_together(content)
        
        if markdown_content is None:
            print("❌ Không thể chuyển đổi sang Markdown")
            failed_conversions += 1
            continue
        
        # Lưu file Markdown
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(markdown_content)
            print(f"✅ Đã lưu thành công: {output_path}")
            successful_conversions += 1
        except Exception as e:
            print(f"❌ Lỗi khi lưu file: {e}")
            failed_conversions += 1
    
    # Báo cáo kết quả
    print(f"\n{'='*60}")
    print("📊 KẾT QUẢ TỔNG KẾT:")
    print(f"✅ Thành công: {successful_conversions} file")
    print(f"❌ Thất bại: {failed_conversions} file")
    print(f"📁 Thư mục output: {output_folder}")
    print(f"{'='*60}")

def check_markdown_files_in_folder(folder_path):
    """Kiểm tra tất cả file Markdown trong folder"""
    result = {}
    folder_path = Path(folder_path)
    
    print(f"\n🔍 Đang kiểm tra file Markdown trong: {folder_path}")
    
    for md_file in folder_path.rglob("*.md"):
        try:
            with open(md_file, 'r', encoding='utf-8') as file:
                content = file.read()
                validation_result = validate_markdown_structure(content)
                result[str(md_file.relative_to(folder_path))] = validation_result
        except Exception as e:
            result[str(md_file.relative_to(folder_path))] = f"Lỗi khi đọc file: {e}"
    
    return result




input_folder = './data'
output_folder  = './markdown'


def main():
    """Hàm chính"""
    print("🚀 DOCX to Markdown Converter với Together AI")
    print("="*60)
    
    # Cấu hình đường dẫn
    input_folder = './data'
    output_folder  = './md_ai'
    
    if not output_folder:
        output_folder = "./md_ai"
    
    # Kiểm tra folder input
    if not os.path.exists(input_folder):
        print(f"❌ Folder không tồn tại: {input_folder}")
        return
    
    # Xử lý chuyển đổi
    process_docx_files(input_folder, output_folder)
    
    # Kiểm tra kết quả
    print("\n🔍 Đang kiểm tra chất lượng file Markdown...")
    validation_results = check_markdown_files_in_folder(output_folder)
    
    if validation_results:
        print("\n📋 KẾT QUẢ KIỂM TRA:")
        for filename, result in validation_results.items():
            print(f"\n📄 File: {filename}")
            if isinstance(result, list):
                for error in result:
                    print(f"  ⚠️  {error}")
            else:
                print(f"  ✅ {result}")
    else:
        print("❌ Không tìm thấy file Markdown nào để kiểm tra")

if __name__ == "__main__":
    main()

🚀 DOCX to Markdown Converter với Together AI
🔍 Đang tìm file DOCX trong: data
📄 Tìm thấy: data/4. Thông tin các khoa/THÔNG TIN CÁC KHOA - VĂN PHÒNG ĐÀO TẠO.docx
📄 Tìm thấy: data/2. Tờ rơi các ngành đào tạo/Mẫu Thông tin về ngành Kinh doanh số.docx
📄 Tìm thấy: data/2. Tờ rơi các ngành đào tạo/QUẢN LÝ CÔNG NGHIỆP.docx
📄 Tìm thấy: data/2. Tờ rơi các ngành đào tạo/KỸ THUẬT VẬT LIỆU.docx
📄 Tìm thấy: data/2. Tờ rơi các ngành đào tạo/PFIEV.docx
📄 Tìm thấy: data/1. Thông tin Tuyển sinh 2025/11.05.25-HCMUT-Thông tin tuyển sinh Đại học chính quy năm 2025.docx
📄 Tìm thấy: data/5. Thông tin PTN-xưởng/Thông tin PTN_Xưởng.docx
📄 Tìm thấy: data/3. Thông tin Lãnh đạo các đơn vị/Thông tin Lãnh đạo các đơn vị.docx
📄 Tìm thấy: data/2. Tờ rơi các ngành đào tạo/CƠ KHÍ/KỸ THUẬT CƠ KHÍ.docx
📄 Tìm thấy: data/2. Tờ rơi các ngành đào tạo/CƠ KHÍ/KỸ THUẬT CƠ ĐIỆN TỬ.docx
📄 Tìm thấy: data/2. Tờ ro

In [13]:
#check markdown
import os
import re

def validate_markdown_structure(content):
    errors = []
    
    # Kiểm tra tiêu đề chính
    if not re.search(r'^#\s+.+', content, re.MULTILINE):
        errors.append("Thiếu tiêu đề chính (H1)")
    
    # Kiểm tra tỷ lệ heading hợp lý
    h2_count = len(re.findall(r'^##\s+', content, re.MULTILINE))
    h3_count = len(re.findall(r'^###\s+', content, re.MULTILINE))
    
    if h3_count > 0 and h2_count == 0:
        errors.append("Có H3 nhưng thiếu H2")
    
    # Kiểm tra code block đóng mở đúng
    code_blocks = re.findall(r'```', content)
    if len(code_blocks) % 2 != 0:
        errors.append("Code block chưa đóng")
    
    return errors if errors else "Cấu trúc hợp lệ"


def check_markdown_files_in_folder(folder_path):
    result = {}
    
    # Duyệt qua tất cả file trong thư mục
    for filename in os.listdir(folder_path):
        if filename.endswith('.md'):  # Kiểm tra nếu file có đuôi .md
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    validation_result = validate_markdown_structure(content)
                    result[filename] = validation_result
            except Exception as e:
                result[filename] = f"Lỗi khi đọc file: {e}"
    
    return result


# Ví dụ sử dụng
folder_path = './md_ai'  # Đổi thành đường dẫn thực tế
validation_results = check_markdown_files_in_folder(folder_path)

for filename, result in validation_results.items():
    print(f"File: {filename}")
    if isinstance(result, list):
        for error in result:
            print(f"  - {error}")
    else:
        print(f"  - {result}")
        


File: 2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT XÂY DỰNG_KỸ THUẬT XÂY DỰNG CÔNG TRÌNH GIAO THÔNG.docx.md
  - Cấu trúc hợp lệ
File: 2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT XÂY DỰNG_KỸ THUẬT VÀ QUẢN LÝ NƯỚC ĐÔ THỊ.docx.md
  - Cấu trúc hợp lệ
File: 2. Tờ rơi các ngành đào tạo_KỸ THUẬT HÓA HỌC_CÔNG NGHỆ SINH HỌC.docx.md
  - Cấu trúc hợp lệ
File: 2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT GIAO THÔNG_KỸ THUẬT Ô TÔ.docx.md
  - Cấu trúc hợp lệ
File: 5. Thông tin PTN-xưởng_Thông tin PTN_Xưởng.docx.md
  - Cấu trúc hợp lệ
File: 2. Tờ rơi các ngành đào tạo_KỸ THUẬT VẬT LIỆU.docx.md
  - Cấu trúc hợp lệ
File: 2. Tờ rơi các ngành đào tạo_CƠ KHÍ_LOGISTIC VÀ HỆ THỐNG CÔNG NGHIỆP.docx.md
  - Cấu trúc hợp lệ
File: 2. Tờ rơi các ngành đào tạo_Mẫu Thông tin về ngành Kinh doanh số.docx.md
  - Cấu trúc hợp lệ
File: 2. Tờ rơi các ngành đào tạo_KỸ THUẬT HÓA HỌC_CÔNG 

In [17]:
import os
from langchain.text_splitter import MarkdownHeaderTextSplitter

# Đường dẫn thư mục đầu vào và đầu ra
input_folder = "./md_ai"  # Thư mục chứa các tệp markdown đã chuyển đổi
output_folder = "./chunked"

# Kiểm tra và tạo thư mục đầu ra nếu chưa có
os.makedirs(output_folder, exist_ok=True)

# Danh sách các tệp markdown trong thư mục đầu vào
input_files = [f for f in os.listdir(input_folder) if f.endswith('.md')]

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# Chunking dựa trên cấu trúc markdown
splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False  # Giữ nguyên tiêu đề trong nội dung
)

# Lặp qua các tệp markdown
for input_file in input_files:
    input_path = os.path.join(input_folder, input_file)
    
    # Đọc nội dung của tệp markdown
    with open(input_path, 'r', encoding='utf-8') as f:
        markdown_content = f.read()

    # Tiến hành chia chunk
    structural_chunks = splitter.split_text(markdown_content)

    # Lưu metadata cấu trúc
    metadata_list = []
    for chunk in structural_chunks:
        metadata = {
            'header': chunk.metadata.get('Header 2', ''),
            'subheader': chunk.metadata.get('Header 3', ''),
            'content_with_markdown': chunk.page_content  # Giữ nguyên markdown
        }
        metadata_list.append(metadata)

    # Tạo tệp output
    output_file = os.path.join(output_folder, f"{os.path.splitext(input_file)[0]}_output.json")
    
    # Lưu metadata dưới dạng tệp JSON (hoặc định dạng bạn muốn)
    with open(output_file, 'w', encoding='utf-8') as out_f:
        import json
        json.dump(metadata_list, out_f, ensure_ascii=False, indent=4)

    print(f"Đã xử lý và lưu tệp {input_file} vào {output_file}")


Đã xử lý và lưu tệp 2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT XÂY DỰNG_KỸ THUẬT XÂY DỰNG CÔNG TRÌNH GIAO THÔNG.docx.md vào ./chunked/2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT XÂY DỰNG_KỸ THUẬT XÂY DỰNG CÔNG TRÌNH GIAO THÔNG.docx_output.json
Đã xử lý và lưu tệp 2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT XÂY DỰNG_KỸ THUẬT VÀ QUẢN LÝ NƯỚC ĐÔ THỊ.docx.md vào ./chunked/2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT XÂY DỰNG_KỸ THUẬT VÀ QUẢN LÝ NƯỚC ĐÔ THỊ.docx_output.json
Đã xử lý và lưu tệp 2. Tờ rơi các ngành đào tạo_KỸ THUẬT HÓA HỌC_CÔNG NGHỆ SINH HỌC.docx.md vào ./chunked/2. Tờ rơi các ngành đào tạo_KỸ THUẬT HÓA HỌC_CÔNG NGHỆ SINH HỌC.docx_output.json
Đã xử lý và lưu tệp 2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT GIAO THÔNG_KỸ THUẬT Ô TÔ.docx.md vào ./chunked/2. Tờ rơi các ngành đào tạo_KHOA KỸ THUẬT GIAO THÔNG_KỸ THUẬT Ô TÔ.docx_output.jso

In [20]:
import os
import pandas as pd
from pathlib import Path

def convert_xlsx_to_csv(source_folder, dest_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    Path(dest_folder).mkdir(parents=True, exist_ok=True)
    
    # Duyệt qua tất cả file trong thư mục nguồn
    for file_name in os.listdir(source_folder):
        if file_name.endswith(('.xlsx', '.xls')):
            file_path = os.path.join(source_folder, file_name)
            
            try:
                # Đọc file Excel
                df = pd.read_excel(file_path, header=0)
                
                # Đổi tên cột (nếu có ít nhất 3 cột)
                new_columns = ['question', 'answer', 'keyword'] + [f'extra_{i}' for i in range(len(df.columns)-3)]
                df.columns = new_columns[:len(df.columns)]
                
                # Tạo tên file CSV
                csv_name = os.path.splitext(file_name)[0] + '.csv'
                csv_path = os.path.join(dest_folder, csv_name)
                
                # Lưu file CSV
                df.to_csv(csv_path, index=False)
                print(f"Đã chuyển đổi: {file_name} -> {csv_name}")
                
            except Exception as e:
                print(f"Lỗi khi xử lý {file_name}: {str(e)}")

if __name__ == "__main__":
    source ='./Phần việc của Hoàng Bùi Nghĩa Dũng'
    dest = './FAQs'
    
    convert_xlsx_to_csv(source, dest)
    print("Hoàn tất quá trình chuyển đổi!")

Đã chuyển đổi: Điện tử - Viễn thông.xlsx -> Điện tử - Viễn thông.csv
Đã chuyển đổi: Kỹ thuật ô tô.xlsx -> Kỹ thuật ô tô.csv
Đã chuyển đổi: Kỹ thuật hàng không.xlsx -> Kỹ thuật hàng không.csv
Đã chuyển đổi: bảo dưỡng công nghiệp.xlsx -> bảo dưỡng công nghiệp.csv
Đã chuyển đổi: PFIEV.xlsx -> PFIEV.csv
Đã chuyển đổi: Kỹ thuật xây dựng công trình biển.xlsx -> Kỹ thuật xây dựng công trình biển.csv
Đã chuyển đổi: Logistics và hệ thống công nghiệp.xlsx -> Logistics và hệ thống công nghiệp.csv
Đã chuyển đổi: Kiến trúc.xlsx -> Kiến trúc.csv
Đã chuyển đổi: Bảng các câu hỏi thường gặp (trừ phần 2).xlsx -> Bảng các câu hỏi thường gặp (trừ phần 2).csv
Đã chuyển đổi: Thiết kế vi mạch.xlsx -> Thiết kế vi mạch.csv
Đã chuyển đổi: Bảng câu hỏi thường gặp(PTN Trọng điểm Quốc gia Điều khiển kỹ thuật số và Kỹ thuật hệ thống).xlsx -> Bảng câu hỏi thư